In [0]:
import tensorflow as tf
!pip install librosa 
import librosa

In [0]:
!ls

hw4_tes.pkl  hw4_trs.pkl  sample_data


In [0]:
from google.colab import files

uploaded = files.upload()

Saving hw4_tes.pkl to hw4_tes.pkl
Saving hw4_trs.pkl to hw4_trs.pkl


In [0]:
# Reading Pickle files and storing them in respective arrays.

import pickle

hw4_trs_pickle_off = open("hw4_trs.pkl","rb")
hw4_trs = pickle.load(hw4_trs_pickle_off)

hw4_tes_pickle_off = open("hw4_tes.pkl","rb")
hw4_tes = pickle.load(hw4_tes_pickle_off)



print("hw4_trs.shape:", hw4_trs.shape)
print("hw4_tes.shape:", hw4_tes.shape)


hw4_trs.shape: (500, 16180)
hw4_tes.shape: (200, 22631)


In [0]:
# Using STFT transform to reshape

import numpy as np

hw4_trs_lib=[]
for i in range(hw4_trs.shape[0]):
    hw4_trs_singleRow=librosa.stft(hw4_trs[i], n_fft=1024, hop_length=512) 
    hw4_trs_lib.append(hw4_trs_singleRow)
    
hw4_tes_lib=[]
for i in range(hw4_tes.shape[0]):
    hw4_tes_singleRow=librosa.stft(hw4_tes[i], n_fft=1024, hop_length=512) 
    hw4_tes_lib.append(hw4_tes_singleRow)

hw4_trs_transpose_intermediate = np.abs(hw4_trs_lib).transpose(0,2,1)
hw4_tes_transpose_intermediate = np.abs(hw4_tes_lib).transpose(0,2,1)

In [0]:
# Train and Test Data Shape

print(np.array(hw4_trs_lib).shape)
print(np.array(hw4_tes_lib).shape)
print(hw4_trs_transpose_intermediate.shape)
print(hw4_tes_transpose_intermediate.shape)

(500, 513, 32)
(200, 513, 45)
(500, 32, 513)
(200, 45, 513)


In [0]:
# Padded 0's and made all the spectograms of equal length. Change shape from [32 , 513] to [45,513]
hw4_trs_transpose_final=[]

for i in range(np.array(hw4_trs_transpose_intermediate).shape[0]):
  
  hw4_trs_transpose_intermediate_pad = np.zeros([45,513], dtype=np.float)

  hw4_trs_transpose_intermediate_pad[:hw4_trs_transpose_intermediate[i].shape[0], :hw4_trs_transpose_intermediate[i].shape[1]] = hw4_trs_transpose_intermediate[i]
  
  hw4_trs_transpose_final.append(hw4_trs_transpose_intermediate_pad)

hw4_trs_transpose = (np.array(hw4_trs_transpose_final))
     
print("hw4_trs_transpose_intermediate.shape:", hw4_trs_transpose_intermediate.shape)                                 
print("Padded vector---> hw4_trs_transpose.shape", hw4_trs_transpose.shape)

hw4_trs_transpose_intermediate.shape: (500, 32, 513)
Padded vector---> hw4_trs_transpose.shape (500, 45, 513)


In [0]:
import itertools as IT
import numpy.random as rng

# Function which will crate 50 batches with "L" as a variable.

def Create_Batch(L, T, d, lib):
  
  hw4_trs_transpose = d
  hw4_trs_lib = hw4_trs_transpose.transpose(0,2,1)
  
 # print("hw4_trs_transpose:", hw4_trs_transpose.shape)
 # print("hw4_trs_lib:", hw4_trs_lib.shape)
  
  rge = int(d.shape[0]/10)
  
  #print("rge:", rge)
  
  for i in range(1,rge):
    
    #if i%10==0:
    
    #  print("======================================================", i , "======================================================")

    # For every speaker, as we have to select L-pairs utterance pairs occurances from the set of 10, so for that using a logic where everytime for every set of 
    # 10 utterances, finding what is the MIN index values and MAX index value of a list and then using that index to generate L different combinations, which am 
    # further using to generate L-pairs utterances.
    
    if i==1:  
		    Mini_50_Batch = []
        
        # multiplying by 10 every value if "i" as there are 10 utterances given for every speaker.
        
		    MaxNbr = i * 10
#		    print("MaxNbr:", MaxNbr)
		    lst = list(range(MaxNbr))
		    
        # Selecting "MIN" and "MAX" index location
		    Min = lst[0]
		    Max = lst[len(lst)-1]+1
    
#		    print("lst:", lst, "and", "Min:", Min, " Max:", Max)
    
  
        # Passing MIN and MAN index location to select the data from the given TRAINING set.
		    All_Utterance_Pairs_Ist_Speaker = np.array(list(IT.combinations(hw4_trs_transpose[Min:Max,:T,:],2)))
      
#		    print(All_Utterance_Pairs_Ist_Speaker.shape)
   
		    # select "L" pairs from All_Utterance_Pairs_Ist_Speaker

		    L_Utterance_Pairs_Ist_Speaker = np.array(list(IT.islice(All_Utterance_Pairs_Ist_Speaker, L)))
		    Positive_Set = L_Utterance_Pairs_Ist_Speaker
    
		    # Randomply sample "L" utterances from the remaining 49 training speaker

		    # print(np.array(hw4_trs_lib).shape)  has (500, 513, 32) shape. So, randomply select "L" indexes 

		    import numpy.random as rng
		    indices = rng.randint(Max,(np.array(hw4_trs_lib).shape)[0],size=(L,))        
        
#		    print("indices:", indices)
    
		    L_Utterance_from_remaining_49_Speaker_utterances =[]

		    for j in indices:
		    		    L_Utterance_from_remaining_49_Speaker_utterances.append(hw4_trs_lib[j])

#		    		    print("L_Utterance_from_remaining_49_Speaker_utterances:", np.array(L_Utterance_from_remaining_49_Speaker_utterances).shape)

		    # "L" utterances from Iterated Speaker
    
		    L_Utterance_from_Ist_Speaker =[]
		    Ist_Indices = rng.randint(Min,Max,size=(L,))

#		    print("Ist_Indices:", Ist_Indices)

		    for k in Ist_Indices:
		    		    L_Utterance_from_Ist_Speaker.append(hw4_trs_lib[k])

#		    		    print("L_Utterance_from_Ist_Speaker:", np.array(L_Utterance_from_Ist_Speaker).shape)

		    Negative_set=[]
		    Negative_set.append(L_Utterance_from_remaining_49_Speaker_utterances)
		    Negative_Set = np.array(Negative_set)
		    Negative_set.append(L_Utterance_from_Ist_Speaker)
		    Negative_Set = np.array(Negative_set).transpose(1,0,3,2)
#		    print("Negative_Set.shape:", Negative_Set.shape)
  
#		    print("All_Utterance_Pairs_Ist_Speaker.shape:",All_Utterance_Pairs_Ist_Speaker.shape)
#		    print("L_Utterance_Pairs_Ist_Speaker.shape:", L_Utterance_Pairs_Ist_Speaker.shape)#
#		    print("Positive_Set.shape:", Positive_Set.shape
#		    print("Negative_Set.shape:", Negative_Set.shape)
    
		    # MiniBatch together with Positive_Set and Negative_Set
		    MiniBatch = []
		    MiniBatch.append(Positive_Set)
		    MiniBatch = np.array(MiniBatch)
#		    print("MiniBatch.shape:::", MiniBatch.shape)
    
		    MiniBatch= list(MiniBatch)
    
		    MiniBatch.append(Negative_Set)
		    MiniBatch = np.array(MiniBatch)
#		    print("MiniBatch.ahape:", MiniBatch.shape)
    
    
    else:
      
		    MaxNbr = i * 10
#		    print("MaxNbr:", MaxNbr)
#		    print("((i-1)*10)", ((i-1)*10))
		    lst = list(range(((i-1)*10),MaxNbr))
		    Min = lst[0]
		    Max = lst[len(lst)-1]+1

#		    print("lst:", lst, "and", "Min:", Min, " Max:", Max)

    
		    All_Utterance_Pairs_Ist_Speaker = np.array(list(IT.combinations(hw4_trs_transpose[Min:Max,:T,:],2)))
#		    print(All_Utterance_Pairs_Ist_Speaker.shape)
   
		    # select "L" pairs from All_Utterance_Pairs_Ist_Speaker

		    L_Utterance_Pairs_Ist_Speaker = np.array(list(IT.islice(All_Utterance_Pairs_Ist_Speaker, L)))
		    Positive_Set = L_Utterance_Pairs_Ist_Speaker

		    # Randomply sample "L" utterances from the remaining 49 training speaker

		    # print(np.array(hw4_trs_lib).shape)  has (500, 513, 32) shape. So, randomply select "L" indexes 


		    indices = rng.randint(Max,(np.array(hw4_trs_lib).shape)[0],size=(L,))
#		    print("indices:", indices)
 
		    L_Utterance_from_remaining_49_Speaker_utterances =[]

		    for j in indices:
		    		    L_Utterance_from_remaining_49_Speaker_utterances.append(hw4_trs_lib[j])

#		    		    print("L_Utterance_from_remaining_49_Speaker_utterances:", np.array(L_Utterance_from_remaining_49_Speaker_utterances).shape)

		    # "L" utterances from Iterated Speaker
    
		    L_Utterance_from_Ist_Speaker =[]
		    Ist_Indices = rng.randint(Min,Max,size=(L,))

#		    print("Ist_Indices:", Ist_Indices)

    
		    for k in Ist_Indices:
		    		    L_Utterance_from_Ist_Speaker.append(hw4_trs_lib[k])
		    		    #print("L_Utterance_from_Ist_Speaker:", np.array(L_Utterance_from_Ist_Speaker).shape)

      
		    Negative_set=[]
		    Negative_set.append(L_Utterance_from_remaining_49_Speaker_utterances)
		    Negative_Set = np.array(Negative_set)
		    Negative_set.append(L_Utterance_from_Ist_Speaker)
		    Negative_Set = np.array(Negative_set).transpose(1,0,3,2)
     
#		    print("All_Utterance_Pairs_Ist_Speaker.shape:",All_Utterance_Pairs_Ist_Speaker.shape)
#		    print("L_Utterance_Pairs_Ist_Speaker.shape:", L_Utterance_Pairs_Ist_Speaker.shape)
#		    print("Positive_Set.shape:", Positive_Set.shape)
#		    print("Negative_Set.shape:", Negative_Set.shape)

    
		    # MiniBatch together with Positive_Set and Negative_Set
		    MiniBatch = []
		    MiniBatch= list(MiniBatch)
		    MiniBatch.append(Positive_Set)
		    MiniBatch = np.array(MiniBatch)
#		    print("MiniBatch.ahape:::", MiniBatch.shape)
    
		    MiniBatch= list(MiniBatch)
    
		    MiniBatch.append(Negative_Set)
		    MiniBatch = np.array(MiniBatch)
#		    print("MiniBatch.ahape:", MiniBatch.shape)
 
    #print("type(MiniBatch):", type(MiniBatch))
    #print("MiniBatch.shape at the end: ", MiniBatch.shape)
    #print("Mini_50_Batch shape is:", np.array(Mini_50_Batch).shape)
  
    Mini_50_Batch.append(MiniBatch)
 #   print("Mini_50_Batch.shape:",np.array(Mini_50_Batch).shape)

  return np.array(Mini_50_Batch)


In [0]:
#L=2
#BATCH_SIZE = L

# FUnction which will be used to create data batches with differnet values of "L"

def get_batch(BATCH_SIZE, T, dataSet, librosaOutput):
  
  Mini_50_Btch = Create_Batch(BATCH_SIZE, T, dataSet, librosaOutput)
  
#  print("Mini_50_Btch.shape:", Mini_50_Btch.shape)
  
  for p in range(2):
    Sp_1= Mini_50_Btch[p][0]
    Sn_2= Mini_50_Btch[p][1]
#    print("Sp_1.shape", Sp_1.shape)
#    print("Sn_2.shape", Sn_2.shape)
  
  # Targets
  Total_BATCH_SIZE = 2*BATCH_SIZE
 # print("Total_BATCH_SIZE:", (Total_BATCH_SIZE))
  
  pairs_targets=np.ones((Total_BATCH_SIZE,))
  pairs_targets[Total_BATCH_SIZE//2:] = 0
    
  targets = [[tt] for tt in pairs_targets]
  
  
  #print("targets:", type(targets), "Sp_1:",Sp_1.shape, "Sn_2", Sn_2.shape)
  
  POS = []
  for g in range(Sp_1.shape[0]):
  #  print("Sp_1[g].shape:", Sp_1[g].shape)
    POS.append(list(Sp_1[g][0]))
    POS.append(list(Sp_1[g][1]))
  
  #print("POS.shape", np.array(POS).shape) 

  NEG = []
  for v in range(Sn_2.shape[0]):
   # print(Sn_2[g].shape)
    NEG.append(list(Sn_2[v][0]))
    NEG.append(list(Sn_2[v][1]))
  
 # print("NEG.shape", np.array(NEG).shape) 

  p=[]

  pos_2=np.array(POS)
  p.append(pos_2)
  #print("p.shape:", np.array(p).transpose(1,2,3,0).shape)
  
  n=[]

  neg_2=np.array(NEG)
  n.append(neg_2)
 # print("n.shape:",np.array(n).transpose(1,2,3,0).shape) 

  return np.array(p).transpose(1,2,3,0), np.array(n).transpose(1,2,3,0), (targets)

  
#pairs1, pairs2 , targets = get_batch( BATCH_SIZE)

#print(pairs1.shape, pairs2.shape, (targets))

** GRAPH - 1**

In [0]:
tf.reset_default_graph()

In [0]:
# Initialising Graph variables

Speak1 = tf.placeholder(dtype=tf.float32, shape=[None, 32, 513,1], name="Speak1")
Speak2 = tf.placeholder(dtype=tf.float32, shape=[None, 32, 513,1], name="Speak2")
label = tf.placeholder(dtype=tf.float32, shape=[None,1], name="label")


In [0]:
# Reshaping input

x_input = tf.reshape(Speak1,[-1,32,513,1]) 

In [0]:
# defining network structure

def siamese_network(x_input, reuse):

    with tf.variable_scope("shared_weights", reuse=reuse):

      conv1 = tf.layers.conv2d(x_input, 16, [4,4],data_format='channels_last', name='conv1', activation=tf.nn.relu)
      maxp1 = tf.layers.max_pooling2d(conv1, (2,2),(1,1), name='maxp1')

      print("conv1", conv1.shape)

      conv2 = tf.layers.conv2d(maxp1, 32, [2,2], name='conv2', data_format='channels_last', activation=tf.nn.relu)
      maxp2 = tf.layers.max_pooling2d(conv2, (2,2),(1,1), name='maxp2')

      print("conv2", conv2.shape)



      flattened = tf.contrib.layers.flatten(conv2)
      dense1 = tf.layers.dense(flattened, 513, activation=tf.nn.sigmoid, name="dense1") 
      
      print("flattened:", flattened.shape)
      print("dense1:", dense1.shape)

        
    return dense1


In [0]:

## Process both inputs, note that the second network re-uses the weights of te first network

out_1 = siamese_network(Speak1, None)
out_2 = siamese_network(Speak2, True) 

conv1 (?, 29, 510, 16)
conv2 (?, 27, 508, 32)
flattened: (?, 438912)
dense1: (?, 513)
conv1 (?, 29, 510, 16)
conv2 (?, 27, 508, 32)
flattened: (?, 438912)
dense1: (?, 513)


In [0]:
l_one = tf.abs(tf.subtract(out_1,out_2))

prediction = tf.layers.dense(l_one, 1, activation=tf.nn.sigmoid, name="output")

mseloss = tf.losses.mean_squared_error(label,prediction)

optimizer = tf.train.AdamOptimizer().minimize(mseloss)



In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

L=40

BATCH_SIZE = L
avg_loss = 0.0
ITER = 2500

T = 32

for iteration in range(ITER):
  
    pairs1, pairs2 , targets = get_batch( BATCH_SIZE, T )

    _, l = sess.run([optimizer, mseloss],    
                          feed_dict={Speak1: pairs1, 
                                     Speak2: pairs2, 
                                     label: targets })
    avg_loss += l/ITER
    
    if (iteration%50) == 0:
      
      print("-----------------------------------------------------------------")
      print("avg_cost:", avg_loss, "and l", l , "and:", "iteration",  iteration)  
      print("-----------------------------------------------------------------")

Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)


/usr/local/lib/python3.6/dist-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


-----------------------------------------------------------------
avg_cost: 5.705568790435791e-05 and l 0.25004798 and: iteration 0
-----------------------------------------------------------------
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Btch.shape: (49, 2, 40, 2, 32, 513)
Mini_50_Bt

** GRAPH - 2, use different loss function**

In [0]:
tf.reset_default_graph()

In [0]:
# Initialising Graph variables

Speak1 = tf.placeholder(dtype=tf.float32, shape=[None, 45, 513,1], name="Speak1")
Speak2 = tf.placeholder(dtype=tf.float32, shape=[None, 45, 513,1], name="Speak2")

label = tf.placeholder(dtype=tf.float32, shape=[None,1], name="label")


In [0]:
# Reshaping input

x_input = tf.reshape(Speak1,[-1,45,513,1]) 

In [0]:
# defining network structure

def siamese_network(x_input, reuse):

    with tf.variable_scope("shared_weights", reuse=reuse):

      conv1 = tf.layers.conv2d(x_input, 16, [4,4],data_format='channels_last', name='conv1', activation=tf.nn.relu)
      maxp1 = tf.layers.max_pooling2d(conv1, (2,2),(1,1), name='maxp1')

      print("conv1", conv1.shape)

      conv2 = tf.layers.conv2d(maxp1, 32, [2,2], name='conv2', data_format='channels_last', activation=tf.nn.relu)
      maxp2 = tf.layers.max_pooling2d(conv2, (2,2),(1,1), name='maxp2')

      print("conv2", conv2.shape)

      flattened = tf.contrib.layers.flatten(conv2)
      dense1 = tf.layers.dense(flattened, 513, activation=tf.nn.sigmoid, name="dense1") 
      
      print("flattened:", flattened.shape)
      print("dense1:", dense1.shape)

        
    return dense1


In [0]:

## Process both inputs, note that the second network re-uses the weights of te first network

Netout_1 = siamese_network(Speak1, None)
Netout_2 = siamese_network(Speak2, True) 

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
conv1 (?, 42, 510, 16)
conv2 (?, 40, 508, 32)

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
flattened: (?, 650240)
dense1: (?, 513)
conv1 (?, 42, 510, 16)
conv2 (?, 40, 508, 32)
flattened: (?, 650240)
dense1: (?, 513)


In [0]:
# Reffered this for Loss as with regular Distance metric, it was not showing good results. https://leimao.github.io/article/Siamese-Network-MNIST/

def contrastive_loss(O1, O2, labels, margin):
    y = tf.reshape(labels, [-1])
    with tf.name_scope("contrastive-loss"):
        l_two = tf.square(O1-O2)
        distance = tf.reduce_sum(l_two, axis=1)
        
        negative = (1-y)*distance
        positive = y*tf.maximum(0.0,margin-distance)
        
        contrastiveloss = negative + positive
        contrastiveloss = tf.reduce_mean(contrastiveloss)
    return contrastiveloss, distance




In [0]:
margin=0.2

contrastiveloss, distance = contrastive_loss(Netout_1, Netout_2, label, margin)

optimizer = tf.train.AdamOptimizer().minimize(contrastiveloss)

Instructions for updating:
Use tf.cast instead.


In [0]:
hw4_trs_transpose.shape

(500, 45, 513)

**First time Run**

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

L=20

BATCH_SIZE = L
avg_loss = 0.0
ITER = 300

T = 32

for iteration in range(ITER):
  
    pairs1, pairs2 , targets = get_batch( BATCH_SIZE, T , hw4_trs_transpose, hw4_trs_lib)

    _, l = sess.run([optimizer, contrastiveloss],    
                          feed_dict={Speak1: pairs1, 
                                     Speak2: pairs2, 
                                     label: targets })
    avg_loss += l/ITER
    
    if (iteration%10) == 0:
      
      print("-----------------------------------------------------------------")
      print("avg_cost:", avg_loss, "and l", l , "and:", "iteration",  iteration)  
      print("-----------------------------------------------------------------")

/usr/local/lib/python3.6/dist-packages/numpy/core/numeric.py:538: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


-----------------------------------------------------------------
avg_cost: 0.0034894204139709474 and l 1.0468261 and: iteration 0
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_cost: 0.4466329749425252 and l 2.5537028 and: iteration 10
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_cost: 0.4954817046721776 and l 0.7764492 and: iteration 20
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_cost: 0.5100802847743034 and l 0.38338763 and: iteration 30
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_cost: 0.5220111457506815 and l 0.33152443 and: iteration 40
-----------------------------------------------------------------
---------------------

In [0]:
avg_loss

0.7064036887884139

**Second time Run**

In [0]:

sess = tf.Session()
sess.run(tf.global_variables_initializer())

L=20

BATCH_SIZE = L
avg_loss = 0.0
ITER = 300

T = 45

for iteration in range(ITER):
  
    pairs1, pairs2 , targets = get_batch( BATCH_SIZE, T , hw4_trs_transpose, hw4_trs_lib)

    _, l = sess.run([optimizer, contrastiveloss],    
                          feed_dict={Speak1: pairs1, 
                                     Speak2: pairs2, 
                                     label: targets })
    avg_loss += l/ITER
    
    if (iteration%10) == 0:
      
      print("-----------------------------------------------------------------")
      print("avg_loss:", avg_loss, "and l", l , "and:", "iteration",  iteration)  
      print("-----------------------------------------------------------------")


-----------------------------------------------------------------
avg_loss: 0.0023238088687260947 and l 0.69714266 and: iteration 0
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_loss: 0.8090353951851527 and l 3.3135345 and: iteration 10
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_loss: 0.8585278954108557 and l 0.6119759 and: iteration 20
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_loss: 0.8698840122918289 and l 0.23871668 and: iteration 30
-----------------------------------------------------------------
-----------------------------------------------------------------
avg_loss: 0.8763453204929829 and l 0.15774068 and: iteration 40
-----------------------------------------------------------------
--------------------

In [0]:
avg_loss

0.9626222522308546

** TEST Data Set**

In [0]:
# Test Data Shape

print(np.array(hw4_tes_lib).shape)

print(hw4_tes_transpose_intermediate.shape)

(200, 513, 45)
(200, 45, 513)


In [0]:
L=20

BATCH_SIZE = L
avg_loss = 0.0
ITER = 10

T = 45
correct_tested = 0

for iteration in range(ITER):
    pairs1, pairs2 , targets = get_batch( BATCH_SIZE, T, hw4_tes_transpose_intermediate, hw4_tes_lib )

    Dstnce = sess.run(distance,    
                          feed_dict={Speak1: pairs1, 
                                     Speak2: pairs2 })
    max_index = np.argmin(Dstnce)
    
    if max_index == 0:
        correct_tested += 1
        
print("Accuracy", correct_tested/ITER)




Accuracy 1.0
